In [ ]:
'''Script for generating paths to minimum number of tetrahedra -- for training word2vec'''
'''
1) search graph for nodes with minimum num tet, save indexes
2) generate shortest paths from remaining nodes to these indexes, and save them as isosigs
'''

#Import Libraries
import regina
import numpy as np
import networkx as nx
import pickle

#Define manifolds
mflds = ['cMcabbgqs','cMcabbjaj','cMcabbgqw','eLAkbcbddhhjhk','cMcabbjqw','gvLQQedfedffrwawrhh','fvPQcdecedekrsnrs','jLvAzQQcfeghighiiuquanobwwr']
mflds_names = ['S3','S2xS1','RP3','L71','L72','T3','PHS','H_SC']

#Select manifolds and depths to run
mfld_idx = 0  
depth_choice = 5

#Import the respective Pachner graph
PG_import = pickle.load(open(f'../Data/PGData/DeepPG/{mflds_names[mfld_idx]}/PG_{mflds_names[mfld_idx]}_({depth_choice},23).pickle', 'rb'))
print(f'Manifold: {mflds_names[mfld_idx]}\nGraph size: {PG_import.number_of_nodes()}')


In [ ]:
#Import respective isosigs
with open(f'../Data/PGData/DeepPG/{mflds_names[mfld_idx]}/T_list_{mflds_names[mfld_idx]}.txt') as file:
    T_list_import = []
    for line in file.readlines()[:PG_import.number_of_nodes()]: 
        T_list_import.append(line.strip('\n'))
#Identify their respective number of tetrahedra
numtet_labels = dict(list(enumerate([regina.engine.Triangulation3.fromSig(sig).countTetrahedra() for sig in T_list_import])))

#Find and return the triangulation isosigs with the minimum number of tetrahedra
min_value = float('inf')
min_keys = []
for key, value in numtet_labels.items():
    if value < min_value:
        min_value = value
        min_keys = [key]
    elif value == min_value:
        min_keys.append(key)

print(f'Keys with minimum value ({numtet_labels[min_keys[0]]}): {min_keys}')
print(f'Respective IsoSigs: {[T_list_import[idx] for idx in min_keys]}')


In [ ]:
#Generate the minimising paths (from random starting points in the Pachner graph to a random minimal triangulation)
#Time the generation
import time
st = time.time()

#Decide how many paths to generate
num_shortestpaths_to_generate = int(1e4)
num_shortestpaths_to_generate = min(num_shortestpaths_to_generate,PG_import.number_of_nodes()) #...can't generate more paths than there are nodes
start_nodes = np.random.choice(PG_import.number_of_nodes(),num_shortestpaths_to_generate,replace=False)

#Generate the paths
paths = []
for idx, sn in enumerate(start_nodes):
    min_node = np.random.choice(min_keys)
    paths.append(nx.shortest_path(PG_import,sn,min_node))
    if idx%100 == 0:
        print(f'{idx}: {time.time()-st}',flush=True)


In [ ]:
#Print some example paths
print(f'Number of paths: {len(paths)}')
for i in np.random.choice(len(paths),5):
    print([T_list_import[node] for node in paths[i]])
    

In [ ]:
#Save these paths as lists of their isosigs
with open(f'../Data/Databases/MinimisingPaths/MinimisingPaths_{mflds_names[mfld_idx]}.txt','w') as file:
    for pp in paths:
        file.write(str([T_list_import[node] for node in pp])+'\n')
